In [2]:
!pip install sentence_transformers

  Using cached sentence_transformers-3.0.1-py3-none-any.whl (227 kB)
  Using cached transformers-4.42.4-py3-none-any.whl (9.3 MB)
  Using cached huggingface_hub-0.23.4-py3-none-any.whl (402 kB)
  Using cached torch-2.3.1-cp39-cp39-manylinux1_x86_64.whl (779.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached tokenizers-0.19.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)


In [5]:
from sentence_transformers import SentenceTransformer

In [6]:
model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

/home/vijay/anaconda3/lib/python3.9/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11070). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.52k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
user_question = "I just discovered the course. Can I still join it?"

In [31]:
v = embedding_model.encode(user_question)


array([ 7.82226399e-02, -4.01311480e-02,  3.86135764e-02, -1.78970673e-04,
        8.92346874e-02, -5.04591279e-02, -1.05026430e-02,  3.71055752e-02,
       -4.18714061e-02,  3.48084643e-02, -1.20701715e-02, -2.36942228e-02,
        3.87900136e-02,  1.60988551e-02,  3.50747444e-02,  3.04743554e-03,
        5.79672270e-02, -4.10627685e-02, -3.41552570e-02, -2.56396532e-02,
       -3.55263948e-02,  1.42907938e-02, -1.62799824e-02,  3.21446583e-02,
       -4.66897674e-02,  7.89185911e-02,  4.90161031e-02,  1.56761073e-02,
       -1.69109963e-02,  2.26482097e-02,  5.60206026e-02, -3.98361459e-02,
        6.77409843e-02, -1.20209809e-02,  1.12621638e-03, -1.94394235e-02,
       -2.65951622e-02,  1.06177563e-02,  1.69687159e-02,  1.13487858e-02,
       -2.97063068e-02,  5.25258034e-02, -1.41453659e-02,  4.61699963e-02,
        1.17065832e-02, -2.38053538e-02, -6.32558018e-02, -1.92041844e-02,
       -7.10589159e-03,  3.24167833e-02,  2.49618236e-02, -5.27498499e-03,
        2.01149192e-02, -

In [10]:
len(v)

768

In [11]:
v[0]

0.07822264

In [78]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [79]:
doc_results = []
for result in documents:
    if result['course'] == "machine-learning-zoomcamp":
        doc_results.append(result)
len(doc_results)

375

In [37]:
from tqdm.auto import tqdm
embeddings = []

for doc in tqdm(doc_results):
        question = doc['question']
        text = doc['text']
        qa_text = f'{question} {text}'
        v_qt = embedding_model.encode(qa_text)
        embeddings.append(v_qt)
    

  0%|          | 0/375 [00:00<?, ?it/s]

In [40]:
import numpy as np
X = np.array(embeddings)
(X.shape)

(375, 768)

In [41]:
scores = X.dot(v)

In [43]:
np.max(scores)

0.6506573

In [80]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=doc_results, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

In [81]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [82]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [60]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total)
    }

In [61]:
def question_text(q):
    question = q['question']

    v_q = embedding_model.encode(question)

    return search_engine.search(v_q, num_results=5)

evaluate(ground_truth, question_text)

  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.9398907103825137}

In [88]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [91]:
from tqdm.auto import tqdm

for doc in tqdm(doc_results):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text
    doc['question_text_vector'] = embedding_model.encode(qt)

  0%|          | 0/375 [00:00<?, ?it/s]

In [92]:
for doc in tqdm(doc_results):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/375 [00:00<?, ?it/s]

In [93]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [94]:
elastic_search_knn(
    'question_text_vector', v , "machine-learning-zoomcamp")

[{'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'},
 {'question': 'I just joined. What should I do next? How can I access course materials?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the vid

In [98]:
def question_text_knn(q):
    question = q['question']
    course = q['course']

    v_q = embedding_model.encode(question)
    

    return elastic_search_knn('question_text_vector', v_q , course)

evaluate(ground_truth, question_text_knn)

  0%|          | 0/1830 [00:00<?, ?it/s]

{'hit_rate': 0.9398907103825137}